In [13]:
import numpy as np
import pandas as pd
import dataread
from dataread import *

In [ ]:
train_df = preprocess(get_train_df())
train_df.head()

In [ ]:
test_df = preprocess(get_test_df())
test_df.head()